In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import sklearn

# Esse jupyter se dividirá em 3 etapas!
* 1º - Colaborative Filtering de Feedback Explícito
* 2º - Colaborative Filtering de Feedback Implícito
* 3º - Modelo Content-Based

In [2]:
dfOrders = pd.read_csv('DNC_orders_dataset.csv')
dfItems = pd.read_csv('DNC_order_items_dataset.csv')
dfReviews = pd.read_csv('DNC_order_reviews_dataset.csv')
dfProducts = pd.read_csv('DNC_products_dataset.csv')

In [3]:
dfReviews.drop(columns='Unnamed: 0', inplace=True)
dfItems.drop(columns='Unnamed: 0', inplace=True)
dfOrders.drop(columns='Unnamed: 0', inplace=True)
dfProducts.drop(columns='Unnamed: 0', inplace=True)

# 1º - Começando pelo modelo de Feedback Explícito!

In [4]:
df1 = pd.merge(dfReviews,dfItems)
df1 = pd.merge(df1,dfOrders)

## Filtrar pelos que mais foram avaliados para reduzir o DataSet (O pandas não aguenta!)

In [5]:
df1_sorted = df1[['product_id','customer_id','review_score']]\
.groupby('product_id')\
.count()\
.sort_values(by='customer_id',ascending=False)
customers = df1_sorted[df1_sorted['customer_id']>10].index
df1 = df1[(df1['order_status']=='delivered')&(df1['product_id'].isin(customers))][['product_id','customer_id','review_score']]

## Matriz de Ratings e Matriz de Similaridade

In [7]:
Matriz_Produtos = df1 \
.pivot_table(index='product_id',columns='customer_id',values='review_score',aggfunc='mean')\
.applymap(lambda x:x if x>0 else 0)

In [8]:
Sim_Produtos = pd.DataFrame(cosine_similarity(Matriz_Produtos),index=Matriz_Produtos.index, columns = Matriz_Produtos.index)

## N Mais próximos

In [9]:
Sim_Produtos['001b72dfd63e9833e8c02742adf472e3'].sort_values(ascending=False)[:10]

product_id
001b72dfd63e9833e8c02742adf472e3    1.000000
e19ddcc85537b41f22116c8d5425ef46    0.098177
aa5d6a9bd4be367ae1a3a29c323c7c11    0.000000
abc3432ff825d8e524db1d4eac6d2099    0.000000
aba86c093ccdbac75b09111d57e50004    0.000000
ab9cf155f8280c3739b09fe341a185e8    0.000000
ab5da1daa941470d14366f4e76a99dd2    0.000000
ab1f9387c0627dd24000bfbc54fdeee0    0.000000
ab1ecbce7afd9f6e661baa24a5706c9c    0.000000
ab1106e227a745836a0dc83b374e2575    0.000000
Name: 001b72dfd63e9833e8c02742adf472e3, dtype: float64

## N mais próximos ordenado por produto (Matriz de Recomendação!)

In [10]:
ItemNeighbours = pd.DataFrame(index=Sim_Produtos.columns,columns=range(1,10))
for i in range(0,len(Sim_Produtos.columns)): 
    ItemNeighbours.iloc[i,:9] = Sim_Produtos.iloc[0:,i].sort_values(ascending=False)[:9].index
ItemNeighbours

,1,2,3,4,5,6,7,8,9
product_id,,,,,,,,,
001b72dfd63e9833e8c02742adf472e3,001b72dfd63e9833e8c02742adf472e3,e19ddcc85537b41f22116c8d5425ef46,aa5d6a9bd4be367ae1a3a29c323c7c11,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2,ab1f9387c0627dd24000bfbc54fdeee0,ab1ecbce7afd9f6e661baa24a5706c9c
00250175f79f584c14ab5cecd80553cd,00250175f79f584c14ab5cecd80553cd,afeeea6271148ee1bb15173b8187c431,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2,ab1f9387c0627dd24000bfbc54fdeee0,ab1ecbce7afd9f6e661baa24a5706c9c
005030ef108f58b46b78116f754d8d38,005030ef108f58b46b78116f754d8d38,51250f90d798d377a1928e8a4e2e9ae1,75c06ee06b201f9b6301d2b5e72993f8,001b72dfd63e9833e8c02742adf472e3,aa8d88eb4b9cb38894e33fa624c4287f,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8
008cff0e5792219fae03e570f980b330,008cff0e5792219fae03e570f980b330,001b72dfd63e9833e8c02742adf472e3,aa70ae9ee9a798d44ae6e00ce6d82cb1,ac6c3623068f30de03045865e4e10089,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2
00ba6d766f0b1d7b78a5ce3e1e033263,00ba6d766f0b1d7b78a5ce3e1e033263,b931645cdc2d9868f01544e8db63f5ab,001b72dfd63e9833e8c02742adf472e3,aa70ae9ee9a798d44ae6e00ce6d82cb1,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2
...,...,...,...,...,...,...,...,...,...
ff5f8606556b625613f069a5a9a48966,ff5f8606556b625613f069a5a9a48966,001b72dfd63e9833e8c02742adf472e3,aa5d6a9bd4be367ae1a3a29c323c7c11,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2,ab1f9387c0627dd24000bfbc54fdeee0
ff7fccf8513f360157f0660fe51d1d88,ff7fccf8513f360157f0660fe51d1d88,001b72dfd63e9833e8c02742adf472e3,aa5d6a9bd4be367ae1a3a29c323c7c11,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2,ab1f9387c0627dd24000bfbc54fdeee0
ffa7e0cbe11656d11a117b534bb1db27,ffa7e0cbe11656d11a117b534bb1db27,001b72dfd63e9833e8c02742adf472e3,aa5d6a9bd4be367ae1a3a29c323c7c11,ac44def10e24998cdee20a208d691127,abc3432ff825d8e524db1d4eac6d2099,aba86c093ccdbac75b09111d57e50004,ab9cf155f8280c3739b09fe341a185e8,ab5da1daa941470d14366f4e76a99dd2,ab1f9387c0627dd24000bfbc54fdeee0


### A Matriz resultante é extremamente esparsa!!!

# 2º - Vamos agora para o modelo de feedback Implícito!

In [11]:
dfImplicito = pd.merge(dfOrders,dfItems)[['product_id','customer_id']]

### Reduzindo o tamanho do dataset pegando apenas os produtos mais comprados

In [12]:
df1_sorted = dfImplicito[['product_id','customer_id']]\
.groupby('product_id')\
.count()\
.sort_values(by='customer_id',ascending=False)
customers = df1_sorted[df1_sorted['customer_id']>9].index
dfImplicito = dfImplicito[dfImplicito['product_id'].isin(customers)]

### Criando uma nova coluna que será usada para fazer o Pivot (Para obter a matriz de compras!)

In [13]:
dfImplicito['compra'] = 1

### Matriz de compras 

In [14]:
Matriz_Produtos = dfImplicito \
.pivot_table(index='product_id',columns='customer_id',values='compra',aggfunc='sum')\
.applymap(lambda x: 1 if x>0 else 0)

### Vamos ver para um determinado cliente, quantos produtos ele comprou!

In [15]:
Matriz_Produtos['00f83de98e791c2a4b779f7e61a4cf28'].sort_values(ascending=False)

product_id
ab40c45cb88545d5efa2679b30cec4c3    1
001b72dfd63e9833e8c02742adf472e3    0
aa70ae9ee9a798d44ae6e00ce6d82cb1    0
ac44def10e24998cdee20a208d691127    0
abc3432ff825d8e524db1d4eac6d2099    0
                                   ..
55782cb82e0efe052da0a3da237da3b2    0
5570e58fb62fc2a83e5bda3974e35be5    0
553e0e7590d3116a072507a3635d2877    0
5526b1ae9ab2688cf600783cece249df    0
ffc9caf33e2d1e9f44e3e06da19085f7    0
Name: 00f83de98e791c2a4b779f7e61a4cf28, Length: 1919, dtype: int64

### A matriz Permanece SUPER esparsa, não vou nem realizar a similaridade de cossenos, nem obter a matriz de recomendações para cada produto.

# 3º - Content Based

In [16]:
dfCB = pd.merge(dfProducts,dfItems).drop(columns=['order_id','order_item_id'])

### Obtendo a matriz de características por produto

In [17]:
matrizProdutosCB = dfCB \
.pivot_table(columns='product_category_name',values=['product_name_lenght',
       'product_description_lenght', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'price'],index = 'product_id', aggfunc= 'mean')\
.applymap(lambda x:x if x>0 else 0)
matrizProdutosCB.columns = ['_'.join(col) for col in matrizProdutosCB.columns]
matrizProdutosCB

,price_agro_industria_e_comercio,price_alimentos,price_alimentos_bebidas,price_artes,price_artes_e_artesanato,price_artigos_de_festas,price_artigos_de_natal,price_audio,price_automotivo,price_bebes,...,product_width_cm_pet_shop,product_width_cm_portateis_casa_forno_e_cafe,product_width_cm_portateis_cozinha_e_preparadores_de_alimentos,product_width_cm_relogios_presentes,product_width_cm_seguros_e_servicos,product_width_cm_sinalizacao_e_seguranca,product_width_cm_tablets_impressao_imagem,product_width_cm_telefonia,product_width_cm_telefonia_fixa,product_width_cm_utilidades_domesticas
product_id,,,,,,,,,,,,,,,,,,,,,
00066f42aeeb9f3007548bb9d3f33c38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00088930e925c41fd95ebfe695fd2655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0009406fd7479715e4bef61dd91f2462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b8f95fcb9e0096488278317764d19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
000d9be29b5207b54e86aa1b1ac54872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff6177642830a9a94a0f2cba5e476d1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff81cc3158d2725c0655ab9ba0f712c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff9553ac224cec9d15d49f5a263411f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Similaridade de cossenos

In [21]:
Sim_Produtos = pd.DataFrame(cosine_similarity(matrizProdutosCB[:10000]),index=matrizProdutosCB[:10000].index, columns = matrizProdutosCB[:10000].index)

In [22]:
Sim_Produtos['00066f42aeeb9f3007548bb9d3f33c38'].sort_values(ascending=False).head(30)

product_id
00066f42aeeb9f3007548bb9d3f33c38    1.000000
0a795de8287e3f8f1f6c2ddcd8091f8e    0.999559
35ad08ea141acf7c484c97075944610f    0.999010
294d41c2e68c3452a16c269c5a969537    0.998714
00baba5b58e274d0332a0c8a0a66f877    0.998656
0226c9207aea0b16076a27ca65cb4929    0.998610
48e1175aec66cc4cd9998ea2e6eeae04    0.998479
20f160a9cf2e856ee5eba259e9ae33f9    0.998150
46957faa8ca08af2cb0e35f42fac6141    0.998049
1cc5f038fa8645580602fb2423d3a50a    0.997756
2f1507d615de7f4f0e73e3be811ee6bc    0.997743
25e531393ced00e48e0c790fb864f1bf    0.997582
0bdc4af29a51151df496d686dbce59a8    0.997230
3488d2ce36e718097c1509444289ef7f    0.997138
1175dc374bce5c9d93c4d4281d8d85d7    0.996900
405b91eb4d099e9985d4c2342381e919    0.996599
2efe857938eed1619e7399bd614e8c38    0.996321
2c28d5cc196e413502b3f8a4ed9aedc9    0.996189
4419a1c234d848b2bc86c53b222499b2    0.996027
013298e0648d412de0d65690045493bc    0.995627
027cdd14a677a5834bc67a9789db5021    0.995524
0af275b6e520e949016aa725797437a5    0.995316

### N Mais Próximos por produto

In [23]:
ItemNeighbours = pd.DataFrame(index=Sim_Produtos.columns,columns=range(1,10))
for i in range(0,len(Sim_Produtos.columns)): 
    ItemNeighbours.iloc[i,:9] = Sim_Produtos.iloc[0:,i].sort_values(ascending=False)[:9].index
ItemNeighbours

,1,2,3,4,5,6,7,8,9
product_id,,,,,,,,,
00066f42aeeb9f3007548bb9d3f33c38,00066f42aeeb9f3007548bb9d3f33c38,0a795de8287e3f8f1f6c2ddcd8091f8e,35ad08ea141acf7c484c97075944610f,294d41c2e68c3452a16c269c5a969537,00baba5b58e274d0332a0c8a0a66f877,0226c9207aea0b16076a27ca65cb4929,48e1175aec66cc4cd9998ea2e6eeae04,20f160a9cf2e856ee5eba259e9ae33f9,46957faa8ca08af2cb0e35f42fac6141
00088930e925c41fd95ebfe695fd2655,00088930e925c41fd95ebfe695fd2655,144eebc150ba34b9f44f761513b5a394,1b1e7f96bdba2a29b52f73065b193c26,343822490e45c4d1d286b343ce41e634,1f49bc80950157ddb33dfad044f20be7,1abacc695f40d21534ff1c657daed7c4,0ad48bfc4a5619823e2b0f24b3c0b11b,17eaf73dce768e344cb5d1210bea1da2,17f1ed2a8b51d7dfdd9fec1ecc1c9abd
0009406fd7479715e4bef61dd91f2462,0009406fd7479715e4bef61dd91f2462,3c332c78cd1032a39727291af7ad2932,10163fabc9375e9bb72a109272730c6b,39f9ab5ea27485a4c3d32ee4c45630d8,3b0fbb3ab5b1512278d2e1c695b07e85,48cd33248ca35fac04a3b751ac966810,3b40b07af370eaeb124725c4c70ed73a,1f58ebdc810e1d5c7cead7f763c27a94,230aa1590e1e7bc88252d8f1649109c2
000b8f95fcb9e0096488278317764d19,000b8f95fcb9e0096488278317764d19,24c9dc6efe88a1c3e794968ab1dd2a38,4a2103f59fe65c15afb3cdcd24a7ff70,2d1a9ea0989c21fc3903ab08be09a4f0,13900a14a66366f79faa0ff07fe354ef,39295d6c2311ab7091b5fc95bb51bbe7,24d95adbf4d5909e2353d0256eb3ab4f,3a0a670620f51d9287a85a9d81474dc0,00b4155166f994ba9da3cf001eb80505
000d9be29b5207b54e86aa1b1ac54872,000d9be29b5207b54e86aa1b1ac54872,1c0b635cee597371e54d222a1d8e0386,4b2e8d0baa7e0e7483c8bfe1dc33e406,3ef588bfd98e3e3a65ab2478b5b32824,4579e87cfe02f4b4eaabb298a254a5b9,425a12d4dd8e5e7ba3be5c4157f1fafc,4cd623b0731519cf74354784255b2b37,327018136fd4a966408df8243383a233,2ffdf10e724b958c0f7ea69e97d32f64
...,...,...,...,...,...,...,...,...,...
4e45c321347cc3336c76fd6350da3445,4e45c321347cc3336c76fd6350da3445,2bd9b51a9ab079e095aca987845d3266,0be79cf589781c512bd91976eadeeff0,33430c5c1027d812b5c62f778e5ee7f7,40a031e5449e42ea991c4ebd5d8639ce,3519403062e217f433e0bbdc52e0b19f,3904e66405ea8ec8d104534682a38c6d,3fe46d73f7527ad59e36c21c3e783041,4757a525c7f6a182f4c63e23686a092d
4e46d4d7bdbc358eee60e44df825dd01,4e46d4d7bdbc358eee60e44df825dd01,0ac4b9dc5e23908abf09e13da3ea1bbb,2bd11da9314c88bf55df5a07251442e4,27e9b49fde45d10aeb0ec54968eff703,262573ecc3b4b8c70a61fbc90671efa4,05b0feccfab2debfdb5744e42bdb6f71,4965fd68b4fa0cab5fca575c3b4f4b73,3c3a35d265e632d286b29a51496858a7,4a6bed9678fe2fa40d7a84063fde6f89
4e52d603138fe095d25d2c44eb9f7c42,4e52d603138fe095d25d2c44eb9f7c42,3803502e6590e7de4f77af1b2cb8d0c4,427a9f5e8a9d47c60adc5ae34e2657d2,08958eb11767450566d8328f3f1b63b0,39295d6c2311ab7091b5fc95bb51bbe7,24d95adbf4d5909e2353d0256eb3ab4f,48390200e49b570705afb0bf9442ee5e,112d3b0704a6ed73f5b7bbbaf59362b5,1169ec485506aa10f38d941275dfce4b


# Código Reutilizável

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import sklearn

dfItems = pd.read_csv('DNC_order_items_dataset.csv')
dfProducts = pd.read_csv('DNC_products_dataset.csv')
dfItems.drop(columns='Unnamed: 0', inplace=True)
dfProducts.drop(columns='Unnamed: 0', inplace=True)
    
class product_check():
    
    def __init__(self, dfItems, dfProducts):
        self.dfCB = pd.merge(dfProducts,dfItems).drop(columns=['order_id','order_item_id'])
    
    def obter_matriz_caracteristicas(self):
        matrizProdutosCB = self.dfCB \
        .pivot_table(columns='product_category_name',values=['product_name_lenght',
               'product_description_lenght', 'product_weight_g', 'product_length_cm',
               'product_height_cm', 'product_width_cm', 'price'],index = 'product_id', aggfunc= 'mean')\
        .applymap(lambda x:x if x>0 else 0)
        matrizProdutosCB.columns = ['_'.join(col) for col in matrizProdutosCB.columns]
        self.matriz = matrizProdutosCB
    
    def obter_similaridade_cossenos(self):
                
        obter_similaridade_cossenos = pd.DataFrame(cosine_similarity(self.matriz[:10000]),
                                    index= self.matriz[:10000].index, 
                                    columns = self.matriz[:10000].index)
        self.matriz1 = obter_similaridade_cossenos
    
    def obter_n_mais_proximos(self, n = 0):
        
        if n == 0:
            ItemNeighbours = pd.DataFrame(index=self.matriz1.columns,columns=range(1,10))
            for i in range(0,len(matriz.columns)): 
                ItemNeighbours.iloc[i,:9] = self.matriz1.iloc[0:,i].sort_values(ascending=False)[:9].index
            return ItemNeighbours
        elif n in self.matriz1.columns:
            return self.matriz1[n].sort_values(ascending=False).head(10)
        else: 
            return 'O produto não existe na lista!'

product_check_objects = product_check(dfItems,dfProducts)
product_check_objects.obter_matriz_caracteristicas()
product_check_objects.matriz

,price_agro_industria_e_comercio,price_alimentos,price_alimentos_bebidas,price_artes,price_artes_e_artesanato,price_artigos_de_festas,price_artigos_de_natal,price_audio,price_automotivo,price_bebes,...,product_width_cm_pet_shop,product_width_cm_portateis_casa_forno_e_cafe,product_width_cm_portateis_cozinha_e_preparadores_de_alimentos,product_width_cm_relogios_presentes,product_width_cm_seguros_e_servicos,product_width_cm_sinalizacao_e_seguranca,product_width_cm_tablets_impressao_imagem,product_width_cm_telefonia,product_width_cm_telefonia_fixa,product_width_cm_utilidades_domesticas
product_id,,,,,,,,,,,,,,,,,,,,,
00066f42aeeb9f3007548bb9d3f33c38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00088930e925c41fd95ebfe695fd2655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0009406fd7479715e4bef61dd91f2462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b8f95fcb9e0096488278317764d19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
000d9be29b5207b54e86aa1b1ac54872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff6177642830a9a94a0f2cba5e476d1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff81cc3158d2725c0655ab9ba0f712c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff9553ac224cec9d15d49f5a263411f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
